### Step 0: Customize
The first code cell in this notebook serves as the central configuration hub, enabling users to customize key parameters that define the behavior of the synthetic data generation workflow. This cell demonstrates several important customization capabilities through its variables:

1. **API Endpoint Customization**  
   - The `SDS_URL` variable allows users to specify the base URL for the Synthetic Data Studio API. This enables easy switching between different service instances or environments.

2. **Model Selection**  
   - The `GenerationInferenceType` and `EvaluationInferenceType` variables let users choose between different inference providers (e.g., CAII or AWS Bedrock) for data generation and evaluation.
   - The `GenerationModelId` and `EvaluationModelId` variables allow specification of different model architectures for generation and evaluation tasks.

3. **Endpoint Configuration**  
   - The `GenerationEndpoint` and `EvaluationEndpoint` variables provide precise control over the specific model endpoints to be used for generation and evaluation tasks.

4. **Path Customization**  
   - Path variables (`Path`, `SDSPath`) enable users to specify where data files are stored and accessed, allowing adaptation to different directory structures or cloud storage configurations.
   - Path defines this notebook path
   - SDSPath defines where the SDS outputs files

5. **Input Data Specification**  
   - The `InputData` variable lets users define the location of the original dataset used for reference in the generation process.

6. In step 2b, the variable needs to be updated with the file output in step 2a

7. In step 3b, LLMJUDGEOUT needs to be replaced with the output of step 3a.

In [28]:
import os
os.environ['SDS_URL']='https://synthetic-data-generator-b2cbtm.ai-workbench.eng-ml-l.vnu8-sqze.cloudera.site/'
#Free form SDS API url
url=os.environ['SDS_URL']+'synthesis/freeform' 
#Free form eval SDS API url
url_eval = os.environ['SDS_URL'] + 'synthesis/evaluate_freeform'
Path='/home/cdsw/Advanced_Tabular_Generation_SDS_Example/'
Path=os.getcwd()
SDSPath='/home/cdsw/'

#Specify the Seed Instructions and examples
SeedsFile='SeedsInstructions.json'
ExamplesData='ExamplesLendingDataSynthetic.json'

#Specify model endpoints
os.environ['GenerationInferenceType']='CAII' #CAII or aws_bedrock
os.environ['EvaluationInferenceType']='CAII' #CAII or aws_bedrock

#Specify the model id and endpoint.
#Ideally use different models. The eval model should be the best model
os.environ['GenerationModelId']="meta/llama-3.1-70b-instruct" 
os.environ['EvaluationModelId']="meta/llama-3.1-70b-instruct" 

os.environ['GenerationEndpoint']="https://caii-prod-long-running.eng-ml-l.vnu8-sqze.cloudera.site/namespaces/serving-default/endpoints/llama-31-70b-instruct-8xl40s/v1/chat/completions" 
os.environ['EvaluationEndpoint']="https://caii-prod-long-running.eng-ml-l.vnu8-sqze.cloudera.site/namespaces/serving-default/endpoints/llama-31-70b-instruct-8xl40s/v1/chat/completions"

#Specify input data. For inspection only. (Not necessary)
InputData='data_tab_separated-large_synthetic.csv'


### Step 1a: Real Data Reference
**Purpose**: Load a sample of the original lending data for understanding the dataset  
**Key Features**:  
- Contains 27 fields including loan amount, interest rate, employment details, and credit history  
- Example fields: `loan_amnt` (loan amount), `int_rate` (interest rate), `emp_title` (job title)  

In [12]:
#These are the lending datasets and how they look like
import pandas as pd
data = pd.read_csv(InputData)
data

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10500.50,36 months,7.85,328.42,A,A4,Project Manager,7 years,MORTGAGE,72000.0,...,11.0,0.0,11500.0,42.50,24.0,w,INDIVIDUAL,1.0,0.0,"123 Oak Street\r\nTulsa, OK 30723"
1,24500.00,60 months,14.85,650.25,C,C4,Operations Manager,7 years,MORTGAGE,95000.0,...,15.0,0.0,28500.0,58.50,33.0,f,INDIVIDUAL,3.0,0.0,"567 Palm Lane\r\nHonolulu, HI 22690"
2,21500.00,36 months,7.82,670.45,A,A4,Financial Analyst,7 years,MORTGAGE,115000.0,...,14.0,0.0,31000.0,48.20,33.0,w,INDIVIDUAL,3.0,0.0,"1234 Palm Street\r\nHonolulu, HI 22690"
3,11975.00,36 months,15.43,378.90,C,C5,Operations Manager,7 years,RENT,62005.0,...,10.0,0.0,10983.0,58.28,21.0,f,INDIVIDUAL,1.0,0.0,"567 Pine Street\r\nSacramento, CA 70466"
4,25500.00,36 months,14.25,875.32,C,C2,Operations Manager,7 years,MORTGAGE,115000.0,...,15.0,0.0,35000.0,62.50,35.0,f,INDIVIDUAL,3.0,0.0,"512 Mountain View Drive\r\nBillings, MT 48052"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10864,10258.32,36 months,7.85,321.45,A,A4,Financial Analyst,7 years,MORTGAGE,71500.0,...,11.0,0.0,12000.0,42.50,24.0,w,INDIVIDUAL,2.0,0.0,"825 Palm Boulevard\r\nHonolulu, HI 22690"
10865,10825.50,36 months,14.85,375.32,C,C1,Project Manager,7 years,RENT,58500.0,...,11.0,0.0,11200.0,58.50,22.0,f,INDIVIDUAL,0.0,0.0,"825 Maple Lane\r\nSpringfield, AP 05113"
10866,18500.00,36 months,11.25,560.42,B,B3,Financial Analyst,7 years,MORTGAGE,98000.0,...,13.0,0.0,25600.0,57.50,31.0,f,INDIVIDUAL,3.0,0.0,"825 Palm Grove Lane\r\nHonolulu, HI 22690"
10867,11250.50,36 months,14.85,389.62,C,C1,Operations Manager,7 years,RENT,58500.0,...,11.0,0.0,12500.0,62.50,23.0,f,INDIVIDUAL,1.0,0.0,"567 Palm Drive\r\nHonolulu, HI 22690"


### Step 1b: Statistical characteristics for Seed Instructions
**Purpose**: Load all programmatically pre-creaated Statistical characteristics  
**Key Features**:  
- Each line contains statistics about the statistical properties of each variable.


In [13]:
import json
import numpy as np

with open(SeedsFile,'r') as f:
    Seeds=json.load(f)

### Step 1c: Construct the seed instrctions loaded statistical information 
**Purpose**: Construct the seed instrctions by combining the pre-created Construct the seed instrctions with additional language to guide the LLM's understanding.
**Key Features**:  
- Subsample the seeds to a smaller set for demonstation purposes


In [14]:
#Seeds=[ "\nHere are the macro statistics and information for each field to generate.\n\n"+"\n".join(i)+"\n\n"+GenerationConsiderations for i in Seeds]
Seeds=[ "\nHere are the macro statistics and information for each field to generate.\n\n"+"\n".join(i) for i in Seeds]


### Step 1d: Seed instruction example 


In [15]:
print(Seeds[0])


Here are the macro statistics and information for each field to generate.

loan_amnt: mean=10721.3, std=683.73, kurtosis=-1.74, skewness=-0.14
term: 36 months=1.0, 60 months=0.0
int_rate: mean=11.33, std=0.09, kurtosis=-0.85, skewness=0.04
installment: mean=350.24, std=21.12, kurtosis=-1.8, skewness=-0.2
grade=B
sub_grade: B1=0.0, B2=0.0, B3=0.92, B4=0.08, B5=0.0
home_ownership: RENT=0.19, MORTGAGE=0.81, OWN=0.0
annual_inc: mean=65999.07, std=1544.98, kurtosis=-1.11, skewness=0.93
verification_status: Not Verified=0.5, Source Verified=0.5, Verified=0.0
issue_d: mean=2023.0, std=0.0, kurtosis=0.0, skewness=0.0
loan_status=Fully Paid
purpose: car=0.0, credit_card=0.2, debt_consolidation=0.8, home_improvement=0.0, house=0.0, major_purchase=0.0, moving=0.0, other=0.0, small_business=0.0
dti: mean=16.68, std=0.92, kurtosis=-1.22, skewness=-0.24
earliest_cr_line: mean=1999.01, std=0.88, kurtosis=-0.63, skewness=-0.39
open_acc: mean=11.06, std=0.29, kurtosis=7.8, skewness=1.84
pub_rec: mean=

In [16]:
print(len(Seeds))

5


### Step 2a: Synthetic Data Generation
**Key Parameters**:
- **Temperature 1.0**: High creativity for diverse outputs
- **max_tokens 8192**: Enough tokens to generate each output fully.
- **Examples**: Uses `ExamplesLoanData.json` for pattern learning
  -  Fields need to match fields in the prompt in the same sequence.
  -  Ensure diversity of examples
  -  Examples are synthetically generated to reduce risk of PII leakage. 
- **Prompt Structure**:  
  - Specifies all 27 fields with precise definitions  
  - Guidelines for formatting, cross-row and cross-column relationships, data distribution, and background knowledge.


In [17]:
%%time
import requests
import os

# Get API key from environment variable if within CDSW app/session
api_key = os.environ.get('CDSW_APIV2_KEY')


# URL for synthesis


# Add the API key to headers with proper Authorization format
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# If API key exists, add it to the headers
if api_key:
    headers['Authorization'] = f'Bearer {api_key}'
else:
    print("Warning: No API key provided")

# Payload for data synthesis
payload = {
  #Use CAII models
  "inference_type": os.environ['GenerationInferenceType'],
  "caii_endpoint": os.environ['GenerationEndpoint'],
  "model_id": os.environ['GenerationModelId'],
  #Use AWS Bedrock models
  #"inference_type": "aws_bedrock",
  #"model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",

  "is_demo": False,
  "num_questions": 2,
  "custom_prompt": """

Create profile data for the LendingClub company which specialises in lending various types of loans to urban customers.

Background:
LendingClub is a peer-to-peer lending platform connecting borrowers with investors. The dataset captures loan applications, 
borrower profiles, and outcomes to assess credit risk, predict defaults, and determine interest rates. 


Loan Record field:

Each generated record must include the following fields in the exact order provided, with values generated as specified:  

- loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department 
  reduces the loan amount, then it will be reflected in this value.
- term: The number of payments on the loan. Values are in months and can be either " 36 months" or " 60 months".
- int_rate: Interest Rate on the loan
- installment: The monthly payment owed by the borrower if the loan originates.
- grade: LC assigned loan grade (Possible values: A, B, C, D, E, F, G)
- sub_grade: LC assigned loan subgrade (Possible sub-values: 1-5 i.e. A5)
- emp_title: The job title supplied by the Borrower when applying for the loan.
- emp_length: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 
  means ten or more years.
- home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report.
  Possible values are: RENT, OWN, MORTGAGE, ANY, OTHER
- annual_inc: The self-reported annual income provided by the borrower during registration.
- verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified
- issue_d: The month which the loan was funded
- loan_status: Current status of the loan (Possible values: "Fully Paid", "Charged Off")
- purpose: A category provided by the borrower for the loan request.
- title: The loan title provided by the borrower
- dti: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage
  and the requested LC loan, divided by the borrower’s self-reported monthly income.
- earliest_cr_line: The month the borrower's earliest reported credit line was opened
- open_acc: The number of open credit lines in the borrower's credit file.
- pub_rec: Number of derogatory public records
- revol_bal: Total credit revolving balance
- revol_util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available 
  revolving credit.
- total_acc: The total number of credit lines currently in the borrower's credit file
- initial_list_status: The initial listing status of the loan. Possible values are: w, f
- application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers
- mort_acc: Number of mortgage accounts.
- pub_rec_bankruptcies: Number of public record bankruptcies
- address: The physical address of the person

In addition to the definitions above, when generating samples, adhere to following guidelines:

Privacy Compliance guidelines:
1) Ensure PII from examples such as addresses are not used in the generated data to minimize any privacy concerns. 
2) Avoid real PII in addresses. Use generic street names and cities.  

Formatting guidelines:
1) Use consistent decimal precision (e.g., "10000.00" for loan_amnt).  
2) Dates (e.g. issue_d, earliest_cr_line) should follow the "Jan-YYYY" format.
3) term has a leading space before the number of months (i.e. " 36 months")
4) The address field is a special case where the State zipcode needs to be exactly as specified in the seed instructions. 
   The persons address must follow the format as specified in the examples with the State zipcode coming last.
5) Any other formatting guidelines that can be inferred from the examples or field definitions but are not listed above.

Cross-row guidelines:
1) Generated data should maintain consistency with all statistical parameters and distributions defined in the seed instruction
   across records (e.g., 60% of `term` as " 36 months").

Cross-column guidelines:
1) Ensure logical and realistic consistency and correlations between variables. Examples include but not limited to:
   a) Grade/Sub-grade consistency: Sub-grade must match the grade (e.g., "B" grade → "B1" to "B5").  
   b) Interest Rate vs Grade/Subgrade relationship: Higher subgrades (e.g., A5) could have higher `int_rate` than lower subgrades (e.g., A3).  
   c) Mortgage Consistency: `mort_acc` should be 1 or more if `home_ownership` is `MORTGAGE`. 
   d) Open Accounts: `open_acc` ≤ `total_acc`.  

Data distribution guidelines:
1) Continuous Variables (e.g., `loan_amnt`, `annual_inc`): Adhere to the mean and standard deviation given in the seed 
  instructions for each variable.
2) Categorical variables (e.g., `term`, `home_ownership`): Use probability distributions given in the seed instructions 
  (e.g. 60% for " 36 months", 40% for " 60 months").
3) Discrete Variables (e.g., `pub_rec`, `mort_acc`): Adhere to value ranges and statistical parameters
   provided in the seed instructions.
4) Any other logical data distribution guidelines that can be inferred from the seed instructions or field definitions 
   and are not specified above. 

Background knowledge and realism guidelines:
1) Ensure fields such as interest rates reflect real-world interest rates at the time the loan is issued.
2) Generate values that are plausible (e.g., `annual_inc` ≤ $500,000 for most `emp_length` ranges).  
3) Avoid unrealistic values (e.g., `revol_util` as "200%" is unrealistic).  


""",
  "model_params": {
    "temperature": 1.0, # range 0-2 tyically, low temperature gives high accuracy, high temperature gives diversity
    "top_p": 1.0,
    "top_k": 500,       
    "max_tokens": 8192
  },
  "use_case": "custom",
  "topics": 
    Seeds,
  "example_path": Path+ExamplesData
}

# Make the POST request
response = requests.post(url, headers=headers, json=payload)

# Display the response
print(response.status_code)
#print(response.json())


200
CPU times: user 39.2 ms, sys: 0 ns, total: 39.2 ms
Wall time: 1.61 s


### Step 2b: Filter data based on expected columns
**Filtering**:  
- Check if all samples have the exact columns as expected
- Remove those with different column names than expected
- Important: Update InputFileBase wit hthe output from 2a


In [21]:
import pandas as pd
import json
import numpy as np
InputFileBase='freeform_data_llama_20250603T210750214_final.json'
ExpectedKeys=['Seeds', 'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'title', 'dti', 'earliest_cr_line', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'application_type', 'mort_acc', 'pub_rec_bankruptcies', 'address']

with open(SDSPath+InputFileBase,'r') as f:
  data_j = json.load(f)
FilteredData=[]
CountTotalData=0
for i in data_j:
    CountTotalData+=1
    if (list(i.keys())==ExpectedKeys)==True:
      FilteredData.append(i)
InputFile=Path+'Filtered_' + InputFileBase
with open(InputFile,'w') as f:
  json.dump(FilteredData,f)

df=pd.DataFrame(FilteredData)
print("Bad formating percentage: "+str(round(100-len(df)/CountTotalData*100,2))+"%")

Bad formating percentage: 7.69%


### Step 2b: Filter data based on inaccurate installment value
**Filtering**:  
- Check if the generated installment value is approximately equal to installment calculated from loan_amount, term, and interest rate. 
- Remove those rows with installment value that differs by more than 1% from the installment calculated from loan_amount, term, and interest rate.(42.2% of data kept)
- For the rest rows, we set the generated installment value equal to the the installment calculated from loan_amount, term, and interest rate for precision.


In [22]:
def calculate_emi(principal, annual_interest_rate, term_months):
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.

    :param principal: Loan amount
    :param annual_interest_rate: Annual interest rate in percentage (e.g., 12 for 12%)
    :param term_months: Loan term in months
    :return: Monthly installment (EMI)
    """
    monthly_interest_rate = annual_interest_rate / (12 * 100)  # Convert annual rate to monthly decimal

    if monthly_interest_rate == 0:
        # No interest case
        emi = principal / term_months
    else:
        emi = (principal * monthly_interest_rate * (1 + monthly_interest_rate) ** term_months) / \
              ((1 + monthly_interest_rate) ** term_months - 1)

    return round(emi,2)

term_values = {' 36 months': 36, ' 60 months': 60, '36 months':36, '60 months':60}
df['term'] = df.term.map(term_values)
CorrectSum=0
Ind=[]

for i in range(df.shape[0]):  
  # Example usage:
  loan_amount = df.loc[i,'loan_amnt'] 
  int_rate = df.loc[i,'int_rate']  
  term = df.loc[i,'term']

  emi = calculate_emi(loan_amount, int_rate, term)
  CorrectFlag=round(np.abs((emi-df.loc[i,'installment'])/emi)*100,0)<=1
  df.loc[i,'installment']=emi
  CorrectSum += int(CorrectFlag)
print("Total correct installments: "+ str(round(CorrectSum/df.shape[0]*100,2))+ "%")

FilteredData=df.to_dict
InputFile= Path+'InterestRate_'+InputFileBase 
df.to_json(InputFile, orient='records',lines=False)

with open(InputFile,'r') as f:
  FilteredData=json.load(f)

df=pd.DataFrame(FilteredData)

Total correct installments: 25.0%


### Step 2c: Synthetic Data Inspection
**Sample Output Analysis**:  
- Generated all 27 fields as the original data
- Co-dependent variables vary as expected (i.e. a lower loan grade pays higher interest on the loan)
- Contains plausible occupations 
- Addresses follow realistic patterns 


In [23]:
df

,Seeds,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,\nHere are the macro statistics and informatio...,10110.13,36,11.23,332.09,B,B1,Marketing Manager,5,MORTGAGE,...,10.00,0.00,12432.49,53.76,25.00,f,INDIVIDUAL,2.00,1.00,"123 Oak St, CA 70466"
1,\nHere are the macro statistics and informatio...,10931.55,36,11.42,360.06,B,B3,Software Engineer,8,RENT,...,11.00,0.00,12033.24,51.75,24.00,f,INDIVIDUAL,0.00,0.00,"456 Pine St, CA 70466"
2,\nHere are the macro statistics and informatio...,10729.41,36,11.53,353.97,B,B4,hairdresser,5,RENT,...,12.00,0.00,12205.55,57.98,25.00,f,INDIVIDUAL,0.00,0.00,"451 West St, Apt 12, Provo UT 83700"
3,\nHere are the macro statistics and informatio...,21806.39,36,17.80,786.17,D,D1,Head Chef,10,RENT,...,13.66,0.00,25130.99,66.06,30.36,f,INDIVIDUAL,2.17,0.00,445 N Florida St Leadville VT 00813
4,\nHere are the macro statistics and informatio...,21806.39,36,17.80,786.17,D,D1,Full Service Account Representative,0,RENT,...,13.66,0.00,25130.99,66.06,30.36,f,INDIVIDUAL,2.17,0.00,102 Vaca Valley Expansion Ext Kissimmee VT 00813
5,\nHere are the macro statistics and informatio...,11892.91,36,7.23,368.47,A,A1,ECONOMIST,5,RENT,...,11.25,0.03,13351.91,38.48,25.23,w,INDIVIDUAL,1.77,0.09,"Main St, Apt 345, OK 30723"
6,\nHere are the macro statistics and informatio...,11932.24,60,7.33,238.13,A,A2,INSURANCE AGT,8,MORTGAGE,...,11.35,0.04,13213.41,38.93,25.13,w,INDIVIDUAL,1.91,0.09,"Oak St, Apt 432, OK 30723"
7,\nHere are the macro statistics and informatio...,11841.24,36,6.89,365.03,A,A4,HUMAN RESOURCE,6,RENT,...,11.07,0.06,12739.91,38.22,25.45,w,INDIVIDUAL,1.62,0.11,"Cherry St, Apt 191, OK 30723"
8,\nHere are the macro statistics and informatio...,12003.44,60,7.29,239.33,A,A3,UNION COORD,4,MORTGAGE,...,11.23,0.03,13269.41,39.38,25.11,w,INDIVIDUAL,1.91,0.09,"Maple St, Apt 453, OK 30723"
9,\nHere are the macro statistics and informatio...,11924.05,36,7.12,368.83,A,A5,CEO,10,MORTGAGE,...,11.32,0.08,13087.19,38.81,25.25,w,INDIVIDUAL,1.88,0.11,"Cedar St, Apt 235, OK 30723"


### Step 2c: Synthetic Data Inspection along with seed instruction 
**Purpose**: Spot-check synthetic data values along with the statistical characteristcs given in the seed instructions.  
**Key Insights**: 
- Values generated appear plausible given the statistical characteristics in the seed instructions.


In [24]:
Seed=df.loc[0,'Seeds']
print(Seed)
df.loc[df['Seeds']==Seed,:]


Here are the macro statistics and information for each field to generate.

loan_amnt: mean=10721.3, std=683.73, kurtosis=-1.74, skewness=-0.14
term: 36 months=1.0, 60 months=0.0
int_rate: mean=11.33, std=0.09, kurtosis=-0.85, skewness=0.04
installment: mean=350.24, std=21.12, kurtosis=-1.8, skewness=-0.2
grade=B
sub_grade: B1=0.0, B2=0.0, B3=0.92, B4=0.08, B5=0.0
home_ownership: RENT=0.19, MORTGAGE=0.81, OWN=0.0
annual_inc: mean=65999.07, std=1544.98, kurtosis=-1.11, skewness=0.93
verification_status: Not Verified=0.5, Source Verified=0.5, Verified=0.0
issue_d: mean=2023.0, std=0.0, kurtosis=0.0, skewness=0.0
loan_status=Fully Paid
purpose: car=0.0, credit_card=0.2, debt_consolidation=0.8, home_improvement=0.0, house=0.0, major_purchase=0.0, moving=0.0, other=0.0, small_business=0.0
dti: mean=16.68, std=0.92, kurtosis=-1.22, skewness=-0.24
earliest_cr_line: mean=1999.01, std=0.88, kurtosis=-0.63, skewness=-0.39
open_acc: mean=11.06, std=0.29, kurtosis=7.8, skewness=1.84
pub_rec: mean=

,Seeds,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,\nHere are the macro statistics and informatio...,10110.13,36,11.23,332.09,B,B1,Marketing Manager,5,MORTGAGE,...,10.0,0.0,12432.49,53.76,25.0,f,INDIVIDUAL,2.0,1.0,"123 Oak St, CA 70466"
1,\nHere are the macro statistics and informatio...,10931.55,36,11.42,360.06,B,B3,Software Engineer,8,RENT,...,11.0,0.0,12033.24,51.75,24.0,f,INDIVIDUAL,0.0,0.0,"456 Pine St, CA 70466"


### Section 3a: LLM-Based Evaluation
**Evaluation Process**:  
1. **Prompt**:
   - Ensure consistency of definitions with the generation prompt
   - Specifies all 27 fields with precise definitions  
   - Guidelines for formatting, cross-row and cross-column relationships, data distribution, and background knowledge.

2. **Scoring**:
   - Specify a scale (1-10 scale in this example):
   - Penalties for mistakes in each guideline category


3. **Parameter tuning**: Use low `temperature` (e.g., 0.1) to prioritize accuracy over diversity.  


In [29]:
import requests
import os
#********************Accessing Application**************************
# Get API key from environment variable if withinin CDSW app/session.
# To get your API key for using outside CDSW app/session follow given link.
# https://docs.cloudera.com/machine-learning/cloud/api/topics/ml-api-v2.html
api_key = os.environ.get('CDSW_APIV2_KEY')


# Below is your application API URL, you can look at swagger documentation for all existing # endpoints for current application
# https://<application-subdomain>.<workbench-domain>/docs--> will take user to swagger documentaion
# Link to application can be found on application details page within CAI Workbench.


# URL for evaluation

# Add the API key to headers with proper Authorization format
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'  # Format as specified in the documentation
}   

# The prompt for evaluation
custom_prompt = """

Evaluate the given data for the LendingClub company which specialises in lending various types of loans to urban customers.

Background:
LendingClub is a peer-to-peer lending platform connecting borrowers with investors. The dataset captures loan applications, 
borrower profiles, and outcomes to assess credit risk, predict defaults, and determine interest rates. 



Each generated record must include the following defined fields in the exact order provided, with values generated.

Record Field Definitions:
- loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department 
  reduces the loan amount, then it will be reflected in this value.
- term: The number of payments on the loan. Values are in months and can be either " 36 months" or " 60 months".
- int_rate: Interest Rate on the loan
- installment: The monthly payment owed by the borrower if the loan originates.
- grade: LC assigned loan grade (Possible values: A, B, C, D, E, F, G)
- sub_grade: LC assigned loan subgrade (Possible sub-values: 1-5 i.e. A5)
- emp_title: The job title supplied by the Borrower when applying for the loan.
- emp_length: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 
  means ten or more years.
- home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report.
  Possible values are: RENT, OWN, MORTGAGE, ANY, OTHER
- annual_inc: The self-reported annual income provided by the borrower during registration.
- verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified
- issue_d: The month which the loan was funded
- loan_status: Current status of the loan (Possible values: "Fully Paid", "Charged Off")
- purpose: A category provided by the borrower for the loan request.
- title: The loan title provided by the borrower
- dti: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage
  and the requested LC loan, divided by the borrower’s self-reported monthly income.
- earliest_cr_line: The month the borrower's earliest reported credit line was opened
- open_acc: The number of open credit lines in the borrower's credit file.
- pub_rec: Number of derogatory public records
- revol_bal: Total credit revolving balance
- revol_util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available 
  revolving credit.
- total_acc: The total number of credit lines currently in the borrower's credit file
- initial_list_status: The initial listing status of the loan. Possible values are: w, f
- application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers
- mort_acc: Number of mortgage accounts.
- pub_rec_bankruptcies: Number of public record bankruptcies
- address: The physical address of the person




In addition to the definitions above, when evaluating the data samples, make sure the data adhere to following guidelines:

Privacy Compliance guidelines:
1) Allow PII reducted addresses to ensure privacy is maintained. Also, ensure the records address's zipcode and state match the given values in the seed instructions.

Formatting guidelines:
1) Check for consistent decimal precision.  
2) Ensure dates (e.g. issue_d, earliest_cr_line) follow the "Jan-YYYY" format.
3) Validate that term has space before the number of months (i.e. " 36 months")
4) State zipcode needs to be exactly as specified in the seed instructions. The persons address must follow the format as specified in the examples with the State zipcode coming last.
5) Any other formatting guidelines that can be inferred from the examples or field definitions but are not listed above.

Cross-column guidelines:
1) Check for logical and realistic consistency and correlations between variables. Examples include but not limited to:
   a) Grade/Sub-grade consistency: Sub-grade must match the grade (e.g., "B" grade → "B1" to "B5" possible subgrades).  
   b) Interest Rate vs Grade/Subgrade relationship: Higher subgrades (e.g., A5) could have higher `int_rate` than lower subgrades (e.g., A3).  
   c) Mortgage Consistency: `mort_acc` should be 1 or more if `home_ownership` is `MORTGAGE`. 
   d) Open Accounts: `open_acc` ≤ `total_acc`.  
   Note: Do not deduct point points based on the installment amount and its relationship with interest rate, loan amount, and term. The relationship has already been verified.

Data distribution guidelines:
1) Check if the generated values are statistically possible and within any ranges given the parameters defined in the seed instructions. 

Background knowledge and realism guidelines:
1) Ensure fields such as interest rates reflect real-world interest rates at the time the loan is issued.
2) Check all generated values if they are plausible given real-world background information.



Scoring Workflow: 
1. Start at 10 points and deduct points for each violation:  
   - Privacy Compliance: -1 points for any violations related to privacy guidelines.
   - Formatting: -1 point for any violations related to formatting inconsistencies.  
   - Cross-column: -4 points for any violations related to Cross-column inconsistencies.  
   - Background knowledge and realism: -1 point for any violations related to Background knowledge and realism inconsistencies. 
     Note: Allow made-up PII information without deducting points.
   - Other violations: -2 points for any other violations, inconsistencies that you detect but are not listed above.
2. Cap score at 1 if any critical errors (e.g., PII leakage, missing fields).  
4. Give a score rating 1-10 for the given data.  If there are more than 9 points to subtract use 1 as the absolute minimum scoring. 
5. List all scoring justifications as list.
"""




# Model parameters
model_params = {
    "temperature": 0.1,
    "top_p": 1.0,
    "top_k": 250,
    "max_tokens": 4096
}

payload = {
    "export_type": "local",
    "display_name": "LendingData",
    "import_path": InputFile,
    "import_type": "local",
    #Use CAII models
    "caii_endpoint": os.environ['EvaluationEndpoint'],
    "model_id": os.environ['EvaluationModelId'],
    #"caii_endpoint": "https://caii-prod-long-running.eng-ml-l.vnu8-sqze.cloudera.site/namespaces/serving-default/endpoints/llama3-2-90b-8xl40s/v1/chat/completions",
    #"model_id": "meta/llama-3.2-90b-vision-instruct",
    "inference_type": os.environ['EvaluationInferenceType'],

    #Use AWS Bedrock models
    #"inference_type": "aws_bedrock",
    #"model_id": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    "examples": [{
          'score':10,
          'justification': '''
1. Privacy Compliance: No PII leakage detected. (No deductions).  
2. Formatting Consistency:  
   - Decimal precision (e.g., "10000.00", "12.05%") is correctly applied.  
   - Dates follow "Jan-YYYY" format.  
   - Term includes a space before the numeric value (e.g., " 36 months").  
   - Zipcode and state alignment adheres to guidelines. (No deductions).  
3. Cross-Column Consistency:  
   - Grade/Subgrade Alignment: Subgrades (A5, B2, C4) align with their respective grades.  
   - Interest Rate vs. Grade/Subgrade: Rates increase with lower grades (e.g., 12.05% for A5 vs. 18.5% for C4).  
   - Mortgage Consistency: `mort_acc` matches `home_ownership` (e.g., MORTGAGE → `mort_acc=1`, OWN → `mort_acc=0`).  
   - Open vs. Total Accounts: `open_acc` ≤ `total_acc` in all records.  
   - No inconsistencies detected. (No deductions).  
4. Background Knowledge/Realism:  
   - Interest rates (12–18.5%) align with real-world lending practices at the issuance date.  
   - Loan terms (36/60 months), employment lengths (0–10 years), and DTI ratios (15–25%) are realistic. (No deductions).  
5. Other Violations: None identified.  

Final Rating: 10/10. The data adheres to all guidelines, with no critical errors or inconsistencies.
'''
      }
   ],

    "use_case": "custom",
    "is_demo": False,
    "custom_prompt": custom_prompt,
    "model_params": model_params
}

responseEval = requests.post(url_eval, headers=headers, json=payload)

# Print the response
print(responseEval.status_code)
print(responseEval.json())


200
{'job_name': 'LendingData_1fca', 'job_id': 'j44d-5o9q-clfe-d5hb'}


## Step 3b: Example LLM-as-a-judge output
- Provides a score and a justification for the score which can be used for further filtering.  
- Shows the two synthetic data samples.  One with overall score 10 and another with score 8.
- Important: Update LLMJUDGEOUT with the output from step 3a


In [34]:
import pandas as pd
import json
##############
#Replace this with the output of the LLM-as-a-judge step
LLMJUDGEOUT=SDSPath+'row_data_llama_20250603T211939377_evaluated.json'
###############
with open(LLMJUDGEOUT,'r') as f:
  data_j = json.load(f)
#print(json.dumps(data_j['evaluated_rows'][0:200],indent=4))

print("\n\n\nExample 1")
print("================\n\n\n")
i=data_j['evaluated_rows'][0]
print("Seed instruction:\n\n"+i['row']['Seeds']+"\n\n\n")
del i['row']['Seeds']
print("Record:\n\n")
print(json.dumps(i['row'],indent=4))
print("\nScore: "+str(i['evaluation']['score']))
print("\nJustification: "+i['evaluation']['justification'])

print("\n\n\nExample 2")
print("================\n\n\n")

i=data_j['evaluated_rows'][1]
print("Seed instruction:\n\n"+i['row']['Seeds']+"\n\n\n")
del i['row']['Seeds']
print("Record:\n\n")
print(json.dumps(i['row'],indent=4))
print("\nScore: "+str(i['evaluation']['score']))
print("\nJustification: "+i['evaluation']['justification'])
print("\n\n\n================\n\n\n")







Example 1



Seed instruction:


Here are the macro statistics and information for each field to generate.

loan_amnt: mean=10721.3, std=683.73, kurtosis=-1.74, skewness=-0.14
term: 36 months=1.0, 60 months=0.0
int_rate: mean=11.33, std=0.09, kurtosis=-0.85, skewness=0.04
installment: mean=350.24, std=21.12, kurtosis=-1.8, skewness=-0.2
grade=B
sub_grade: B1=0.0, B2=0.0, B3=0.92, B4=0.08, B5=0.0
home_ownership: RENT=0.19, MORTGAGE=0.81, OWN=0.0
annual_inc: mean=65999.07, std=1544.98, kurtosis=-1.11, skewness=0.93
verification_status: Not Verified=0.5, Source Verified=0.5, Verified=0.0
issue_d: mean=2023.0, std=0.0, kurtosis=0.0, skewness=0.0
loan_status=Fully Paid
purpose: car=0.0, credit_card=0.2, debt_consolidation=0.8, home_improvement=0.0, house=0.0, major_purchase=0.0, moving=0.0, other=0.0, small_business=0.0
dti: mean=16.68, std=0.92, kurtosis=-1.22, skewness=-0.24
earliest_cr_line: mean=1999.01, std=0.88, kurtosis=-0.63, skewness=-0.39
open_acc: mean=11.06, std=0.29, kurtosi

## Step 3c: Filtering and Conversion to Tabular Data
- Filter low-quality rows and convert valid data into a DataFrame.  
- Choose a score threshold (e.g., >9) to balance quality and quantity based on business needs.  


In [35]:
Threshold=8
with open(LLMJUDGEOUT,'r') as f:
  data_j = json.load(f)
TotalSamples=0
AllRows=[]
counter=0
for i in data_j['evaluated_rows']:
  TotalSamples+=1
  if i['evaluation']['score'] >= Threshold:
      del i['row']['Seeds']
      AllRows.append(i['row'])
  counter+=1
import pandas as pd
df=pd.DataFrame.from_records(AllRows)
print("Total Input Samples: " + str(TotalSamples))
print('Number of samples (higher or equal to '+str(Threshold)+ ' evaluation threshold): ' + str(len(df)))


Total Input Samples: 12
Number of samples (higher or equal to 8 evaluation threshold): 12


## Step 4: Saving to CSV
- **Export**: Save the cleaned data to a CSV file for downstream use.  
- **Format consistency**: Use a delimiter like `\t` to avoid conflicts with existing data (e.g., commas in addresses).  


In [36]:
OutputFile='data_tab_separated.csv'
df.to_csv(OutputFile, sep='\t',index=False)
